In [1]:
import folium
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter("ignore")

# Workflow

In [2]:
farmer_count = pd.read_csv(r"F:\DSLR\Folium\Animate\Projects\Delivered\farmer_count_updated.csv")
district_desc = pd.read_csv(r"F:\DSLR\Folium\Animate\Projects\Delivered\District_desc.csv")
service_served = pd.read_csv(r"F:\DSLR\Folium\Animate\Projects\Delivered\districtwise_service_count.csv")

In [3]:
farmer_count.head(2)

,District,num_farmer,num_loan
0,Kurigram,25897,1263
1,Lalmonirhat,23628,715


In [4]:
district_desc.head(2)

,Division,District,area,All_holdings,Non-Firm\nholdings,Total_firm_holdings_no,Small_firm_holdings_no,Medium_firm_holdings_no,Largefirm_holdings_no,Owned_holdings_no,...,Goat,Sheep,Pigs,Horse,Chicken,Ducks,Pigeons,Quails,Turkeys,All Fruits
0,Rajshahi,Natore,515417,451011,196515,254496,225265,26848,2383,324700,...,462459,11551.0,620,166,2467053,1060026,440681,10199,12764,4425.74
1,Rajshahi,Rajshahi,604966,536710,224427,312283,292091,19301,891,387035,...,708868,38013.0,2140,226,1870196,3357486,551084,13869,16715,7346.62


In [5]:
service_served.head(2)

,District,Input_Served,Input_Farmer,Output_Served,Output_Farmer,Advisory_Served,Advisory_Farmer
0,Jessore,446,113,13006,4825,970,762
1,Dhaka,0,0,0,0,0,0


In [6]:
def get_min_max_radius(inner_circle, farmer_count, service_served):
    if inner_circle == 'Registered_Farmer':
        sort_service, radius_feature, tt_feature = farmer_count, 'num_farmer', 'num_loan'
    elif inner_circle == 'Loan':
        sort_service, radius_feature, tt_feature = farmer_count, 'num_loan', 'num_farmer'
    elif inner_circle == 'Input_Served':
        sort_service, radius_feature, tt_feature = service_served, 'Input_Served', 'Input_Farmer'
    elif inner_circle == 'Input_Farmer':
        sort_service, radius_feature, tt_feature = service_served, 'Input_Farmer', 'Input_Served'
    elif inner_circle == 'Output_Served':
        sort_service, radius_feature, tt_feature = service_served, 'Output_Served', 'Output_Farmer'
    elif inner_circle == 'Output_Farmer':
        sort_service, radius_feature, tt_feature = service_served, 'Output_Farmer', 'Output_Served'
    elif inner_circle == 'Advisory_Served':
        sort_service, radius_feature, tt_feature = service_served, 'Advisory_Served', 'Advisory_Farmer'
    elif inner_circle == 'Advisory_Farmer':
        sort_service, radius_feature, tt_feature = service_served, 'Advisory_Farmer', 'Advisory_Served'
        
    sorted_service = sort_service[sort_service[radius_feature] > 0].reset_index(drop=True)
    inner_rad_max, inner_rad_min = sorted_service[radius_feature].max(), sorted_service[radius_feature].min()
    
    return sorted_service, radius_feature, tt_feature, inner_rad_max, inner_rad_min


def spellchecker(dataframe):
    district_mapping = {
        'Bogura': 'Bogra',
        'Jashore': 'Jessore',
        'Netrokona': 'Netrakona',
        'Barishal': 'Barisal',
        'Cumilla': 'Comilla',
        'Brahmanbaria': 'Brahamanbaria',
        'Chapai Nawabganj': 'Nawabganj',
        'Chapainawabganj': 'Nawabganj',
        'Jhalokathi': 'Jhalokati',
        'Moulvibazar': 'Maulvibazar'
    }

    dataframe = dataframe.applymap(lambda x: district_mapping.get(x, x))
    
    return dataframe

In [7]:
inner_circle = 'Registered_Farmer' # Loan, Input_Served, Input_Farmer, Output_Served, Output_Farmer, Advisory_Served, Advisory_Farmer
div = 'All'
dist_name = 'All' 
percentile = 1

if inner_circle in ['Loan', 'Registered_Farmer']:
    farmer_count = spellchecker(farmer_count)
sort_service, radius_feature, tt_feature, inner_rad_max, inner_rad_min = get_min_max_radius(inner_circle, farmer_count,
                                                                                            service_served)

In [8]:
sort_service.head(1)

,District,num_farmer,num_loan
0,Kurigram,25897,1263


In [9]:
radius_feature, tt_feature, inner_rad_max, inner_rad_min

('num_farmer', 'num_loan', 25897, 1)

In [10]:
outer_rad_max, outer_rad_min = district_desc.Total_firm_holdings_no.max(), district_desc.Total_firm_holdings_no.min()

In [11]:
outer_rad_max, outer_rad_min

(689391, 59633)

In [12]:
def df_process(district_desc, sort_service, div, dist):
    if div != "All":
        if dist != "All":
            sort_district_desc = district_desc[(district_desc.Division == div) &
                                               (district_desc.District == dist)].reset_index(drop = True)
        else:
            sort_district_desc = district_desc[district_desc.Division == div].reset_index(drop=True)
    else:
        sort_district_desc = district_desc
        
    sort_service = sort_service[sort_service.District.isin(sort_district_desc.District)].reset_index(drop=True)
    
#     print(len(sort_district_desc), len(sort_service))
    return sort_district_desc, sort_service

In [13]:
tt_crp = "Cultivated_area"
    
if percentile != 1:
    district_desc = district_desc[district_desc.Cultivated_area > district_desc.Cultivated_area.quantile(1-percentile)].reset_index(drop = True)

sort_district_desc, sort_service = df_process(district_desc, sort_service, div, dist_name)

In [14]:
sort_district_desc.head(2)

,Division,District,area,All_holdings,Non-Firm\nholdings,Total_firm_holdings_no,Small_firm_holdings_no,Medium_firm_holdings_no,Largefirm_holdings_no,Owned_holdings_no,...,Goat,Sheep,Pigs,Horse,Chicken,Ducks,Pigeons,Quails,Turkeys,All Fruits
0,Rajshahi,Natore,515417,451011,196515,254496,225265,26848,2383,324700,...,462459,11551.0,620,166,2467053,1060026,440681,10199,12764,4425.74
1,Rajshahi,Rajshahi,604966,536710,224427,312283,292091,19301,891,387035,...,708868,38013.0,2140,226,1870196,3357486,551084,13869,16715,7346.62


In [15]:
sort_service.head(2)

,District,num_farmer,num_loan
0,Kurigram,25897,1263
1,Lalmonirhat,23628,715


In [16]:
mapObj_dist = folium.Map(location=[23.8103, 90.4125], zoom_start=7, attr='Google', tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}')

In [17]:
mapObj_dist

In [18]:
def get_dist_centroid(dst_name):
    district = {'Bagerhat': [22.369779967952407, 89.73956431663815], 'Bandarban': [21.807117016617926, 92.36382543114077],
         'Barguna': [22.14112658246037, 90.12048035991266], 'Barisal': [22.811441437980307, 90.341672338065],
         'Bhola': [22.335697321448244, 90.73334082882661], 'Bogra': [24.82445808503599, 89.37879178145272],
         'Brahamanbaria': [23.952995668273562, 91.0834843288899], 'Chandpur': [23.27356444163652, 90.77101383633664],
         'Chittagong': [22.443264467303408, 91.83436892134053], 'Chuadanga': [23.60831228008582, 88.84797495226135],
         'Comilla': [23.436873149896083, 91.03395508743048], "Cox's Bazar": [21.483797928881796, 92.06392540049595],
         'Dhaka': [23.786967628854637, 90.25076984261663], 'Dinajpur': [25.62987903516547, 88.78688740430421],
         'Faridpur': [23.478941864810682, 89.83887938084165], 'Feni': [22.999431623650427, 91.41243053736657],
         'Gaibandha': [25.298722173106533, 89.50584502438771], 'Gazipur': [24.098424285871662, 90.4449979279924],
         'Gopalganj': [23.10553279678152, 89.89951162322917], 'Habiganj': [24.36808221436504, 91.43033649777905],
         'Jamalpur': [24.973107751934755, 89.84626565885708], 'Jessore': [23.08977613289776, 89.17467566725811],
         'Jhalokati': [22.572786585611468, 90.1840125886512], 'Jhenaidah': [23.48921633521947, 89.08748839319315],
         'Joypurhat': [25.09276871061273, 89.08291551258279], 'Khagrachhari': [23.1643341110586, 91.95214377069831],
         'Khulna': [22.470878221683037, 89.44784775723758], 'Kishoreganj': [24.376186087126992, 90.94240511991032],
         'Kurigram': [25.79209267963356, 89.69470330651363], 'Kushtia': [23.922966848930944, 89.02193154328748],
         'Lakshmipur': [22.891768838589908, 90.85582883101213], 'Lalmonirhat': [26.063112327832865, 89.23573039485389],
         'Madaripur': [23.2180421594776, 90.16509700200096], 'Magura': [23.44317953348163, 89.43371239160088],
         'Manikganj': [23.843925733451535, 89.94959574811202], 'Maulvibazar': [24.48042111032197, 91.91537556818442],
         'Meherpur': [23.794101557216646, 88.70714249349287], 'Munshiganj': [23.525748249240962, 90.41728519420289],
         'Mymensingh': [24.699094214247836, 90.4286088471071], 'Naogaon': [24.899642309556263, 88.75157168481434],
         'Narail': [23.130099524026132, 89.58017588001104], 'Narayanganj': [23.723217645637828, 90.57800728885776],
         'Narsingdi': [24.002290770515735, 90.77323161781602], 'Natore': [24.38036728752842, 89.08685859538868],
         'Nawabganj': [24.715698785557617, 88.26411261626974], 'Netrakona': [24.87076045647337, 90.84462979207186],
         'Nilphamari': [26.0246551122935, 88.92952310037872], 'Noakhali': [22.693696873201098, 91.12465745986425],
         'Pabna': [24.053171551500576, 89.38533226405329], 'Panchagarh': [26.285129105220967, 88.57545903384731],
         'Patuakhali': [22.1908556164555, 90.3989978942543], 'Pirojpur': [22.533792803907883, 89.99318004729419],
         'Rajbari': [23.727655125432264, 89.56152666759257], 'Rajshahi': [24.46574025596341, 88.65143988466103],
         'Rangamati': [22.829978655780568, 92.2795571783611], 'Rangpur': [25.649758375584675, 89.23681345297177],
         'Satkhira': [22.382241376305565, 89.13669129289408], 'Shariatpur': [23.247365088415904, 90.40837734289578],
         'Sherpur': [25.082266685632405, 90.07447787605085], 'Sirajganj': [24.39162970328708, 89.59968044221507],
         'Sunamganj': [24.938567398430816, 91.34604543207142], 'Sylhet': [24.919218341539008, 91.98733437309093],
         'Tangail': [24.356303552452246, 89.99860550188687], 'Thakurgaon': [25.990560117722456, 88.34556890146895]}
    return district[dst_name]
def get_inner_radius(service_type, service_count, max_service, min_service):
    
    if service_type == 'Registered_Farmer':
        px_min, px_max = (8000, 9000) if service_count >= 20000 else \
                         (6000, 7800) if service_count > 10000 else \
                         (5000, 6000) if service_count > 5000 else \
                         (4000, 5000) if service_count > 3000 else \
                         (3500, 4000) if service_count > 2000 else \
                         (3000, 3500) if service_count > 1000 else \
                         (1000, 1800) if service_count > 100 else \
                         (50, 200)
    
    elif service_type == 'Loan':
        px_min, px_max = (8000, 9000) if service_count >= 2000 else \
                         (7500, 7800) if service_count > 1500 else \
                         (6800, 7200) if service_count > 1000 else \
                         (5000, 6000) if service_count > 500 else \
                         (3500, 4500) if service_count > 200 else \
                         (3000, 3400) if service_count > 100 else \
                         (1000, 1800) if service_count > 50 else \
                         (100, 600)
    
    elif service_type == 'Input_Served':
        px_min, px_max = (8000, 9000) if service_count > 5000 else \
                         (5000, 6000) if service_count > 1000 else \
                         (4000, 5000) if service_count > 400 else \
                         (2000, 3000) if service_count > 100 else \
                         (50, 100)
    
    elif service_type == 'Input_Farmer':
        px_min, px_max = (8000, 9000) if service_count > 2000 else \
                         (7000, 7500) if service_count > 1000 else \
                         (5800, 6500) if service_count > 500 else \
                         (3000, 4000) if service_count > 100 else \
                         (500, 1000)
        
    elif service_type == 'Output_Served':
        px_min, px_max = (8000, 9000) if service_count > 10000 else \
                         (7000, 7800) if service_count > 5000 else \
                         (4000, 5000) if service_count > 2000 else \
                         (2000, 3200) if service_count > 100 else \
                         (50, 100)
        
    elif service_type == 'Output_Farmer':
        px_min, px_max = (8500, 9000) if service_count > 5000 else \
                         (7200, 8200) if service_count > 4000 else \
                         (6000, 6800) if service_count > 3000 else \
                         (5000, 5800) if service_count > 2000 else \
                         (4000, 4800) if service_count > 1000 else \
                         (3500, 3800) if service_count > 500 else \
                         (2700, 3200) if service_count > 100 else \
                         (50, 100)
    
    elif service_type == 'Advisory_Served':
        px_min, px_max = (8000, 9000) if service_count > 2000 else \
                         (7000, 7500) if service_count > 1000 else \
                         (6000, 6800) if service_count > 500 else \
                         (4000, 4800) if service_count > 200 else \
                         (200, 1000)
    
    elif service_type == 'Advisory_Farmer':
        px_min, px_max = (8000, 9000) if service_count > 1000 else \
                         (6000, 7800) if service_count > 700 else \
                         (4800, 5800) if service_count > 400 else \
                         (3000, 4200) if service_count > 100 else \
                         (500, 900)
        
    final_rad = int(px_min + ((service_count -min_service)*(px_max - px_min)/(max_service-min_service)))
    return final_rad

In [19]:
np.array(sorted(farmer_count.num_farmer.unique()))

array([    0,     1,     2,     3,     5,     6,     8,    10,    13,
          27,    33,    48,    54,   128,   135,   464,   504,   578,
         584,  1073,  1426,  1476,  1494,  1592,  1806,  1871,  2164,
        2248,  2297,  2554,  2689,  3361,  3836,  4463,  5224,  9201,
       23628, 25897], dtype=int64)

In [20]:
for i in range(len(sort_service)):
        centr = get_dist_centroid(sort_service.District[i])
        
        rad = get_inner_radius(inner_circle, sort_service[radius_feature][i], inner_rad_max, inner_rad_min)
        folium.Circle(location = centr,
                      radius = rad,
                      color ='green',
                      weight =2,
                      fill_color ='green',
                      fill_opacity = 0.9,
                      tooltip = str(sort_service[radius_feature][i]),
                      stroke = True).add_to(mapObj_dist)

In [21]:
mapObj_dist

In [22]:
def get_tt(upz, area, fh, crp, tca, crp_area, reg_far, inner_circle, tt_feature, f_num_tt, prcnt_2):
    
    tt_feature = 'Registered Farmer' if tt_feature == 'num_farmer' else 'Loan Count' if tt_feature == 'num_loan' else tt_feature
    inner_circle = 'Loan Count ' if inner_circle == 'Loan' else 'Registered Farmer' if inner_circle == 'Registered_Farmer' else inner_circle
    f_num_tt_2 = f"{int(f_num_tt):,}"
    tooltip_html = """
        <div style="font-size: 14px">
        <b>District: </b> {upz}
        <br>
        <b>Firm Holdings: </b> {fh}
        <br>
        <b>Area: </b> {area} acre
        <br>
        <b>{crp}: </b> {crp_area} acre
        <br>
        <b>Crop Area: </b> {tca} acre
        <br>
        <b>{tt_feature}: </b> {f_num_tt_2}
        <br>
        <b>{inner_circle}: </b> {reg_far}
        """.format(upz=upz, area=area, fh=fh, crp=crp, tca=tca, crp_area=crp_area, reg_far=reg_far,
                   tt_feature=tt_feature, f_num_tt_2=f_num_tt_2, inner_circle=inner_circle)

    if f_num_tt > 0:
        tooltip_html = tooltip_html + " ({prcnt_2}%)</div>".format(prcnt_2=prcnt_2)
    else:
        tooltip_html = tooltip_html + "</div>"

    return tooltip_html, inner_circle

def circle_plot(centr, param, l_max, l_min, clr, clr_2, opacity_input, tooltip):
    
    shapesLayer = folium.FeatureGroup(name="Revenue")
#     shapesLayer_2 = folium.FeatureGroup(name="Cntre")

    rad = get_rad(param, l_max, l_min)
#     print(rad)
    folium.Circle(location= centr,
                  radius= rad,
                  color=clr,
                  weight=2,
                  fill_color=clr_2,
                  fill_opacity= opacity_input,
                  tooltip = tooltip,
                  stroke = True).add_to(shapesLayer)
    return shapesLayer

def get_rad(loan = 0, l_max = 0, l_min = 0):
    
    px_min = 7000
    px_max = 25000
    
    final_rad = int(px_min + ((loan -l_min)*(px_max - px_min)/(l_max-l_min)))
    return final_rad

In [23]:
sort_district_desc.head(2)

,Division,District,area,All_holdings,Non-Firm\nholdings,Total_firm_holdings_no,Small_firm_holdings_no,Medium_firm_holdings_no,Largefirm_holdings_no,Owned_holdings_no,...,Goat,Sheep,Pigs,Horse,Chicken,Ducks,Pigeons,Quails,Turkeys,All Fruits
0,Rajshahi,Natore,515417,451011,196515,254496,225265,26848,2383,324700,...,462459,11551.0,620,166,2467053,1060026,440681,10199,12764,4425.74
1,Rajshahi,Rajshahi,604966,536710,224427,312283,292091,19301,891,387035,...,708868,38013.0,2140,226,1870196,3357486,551084,13869,16715,7346.62


In [24]:
sort_service.head(2)

,District,num_farmer,num_loan
0,Kurigram,25897,1263
1,Lalmonirhat,23628,715


In [25]:
def get_legend(inner_circle):
    legend =  '''<div style="position: fixed; 
                bottom: 50px; left: 50px; width: 220px; height: 120px; 
                border:2px solid grey; z-index:9999; font-size:14px; background-color: white; display: flex};
                ">
                &nbsp; Description <br>
                &nbsp; <i class="fa fa-circle" style="color:blue"></i> &nbsp; ABC > 100<br>
                &nbsp; <i class="fa fa-circle" style="color:purple"></i> &nbsp; ABC < 100<br>
                &nbsp; <i class="fa fa-circle" style="color:red"></i> &nbsp; ABC 0<br>
                </div>
                '''
    legend = legend.replace('ABC', inner_circle)
    return legend

In [26]:
for i in range(len(sort_district_desc)):
    if (sort_district_desc.District[i] in list(sort_service.District)):
        f_num = sort_service[sort_service.District == sort_district_desc.District[i]].reset_index(drop=True)[radius_feature][0]
        if f_num > 100:
            color = "blue"
        else:
            color = "purple"
    else:
        f_num = 0
        color = 'red'
    try:
        f_num_tt = service_served[service_served.District == sort_district_desc.District[i]].reset_index(drop=True)[tt_feature][0]
    except:
        f_num_tt = farmer_count[farmer_count.District == sort_district_desc.District[i]].reset_index(drop=True)[tt_feature][0]
    
#     print(f_num_tt, type(f_num_tt))
    tt_main, legend_tt = get_tt(sort_district_desc.District[i], f"{int(int(sort_district_desc.area[i])):,}",
                                    f"{int(sort_district_desc.Total_firm_holdings_no[i]):,}",tt_crp,
                                    f"{int(sort_district_desc.Temporary_crops_net_area[i]):,}",
                                    f"{int(sort_district_desc[tt_crp][i]):,}", f"{int(f_num):,}", inner_circle, 
                                    tt_feature, f_num_tt, round((f_num/f_num_tt*100),3))
    tooltip = folium.Tooltip(tt_main)


    centr = get_dist_centroid(sort_district_desc.District[i])

    shapesLayer = circle_plot(centr, sort_district_desc.Total_firm_holdings_no[i]**1.025,
                              outer_rad_max, outer_rad_min, color, color, 0.01, tooltip)
    shapesLayer.add_to(mapObj_dist)

In [27]:
mapObj_dist

In [28]:
legendHtml = get_legend(legend_tt)
mapObj_dist.get_root().html.add_child(folium.Element(legendHtml))

In [29]:
mapObj_dist

In [30]:
def init(inner_circle, district_desc, farmer_count, service_served, div, dist_name, percentile):
    
    if inner_circle in ['Loan', 'Registered_Farmer']:
        farmer_count = spellchecker(farmer_count)
    sort_service, radius_feature, tt_feature, inner_rad_max, inner_rad_min = get_min_max_radius(inner_circle, farmer_count, 
                                                                                    service_served)
    
    outer_rad_max, outer_rad_min = district_desc.Total_firm_holdings_no.max(), district_desc.Total_firm_holdings_no.min() 
    
    tt_crp = "Cultivated_area"
    
    if percentile != 1:
        district_desc = district_desc[district_desc.Cultivated_area > district_desc.Cultivated_area.quantile(1-percentile)].reset_index(drop = True)

    sort_district_desc, sort_service = df_process(district_desc, sort_service, div, dist_name)

    mapObj_dist = folium.Map(location=[23.8103, 90.4125], zoom_start=7, attr='Google', tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}')
    
    for i in range(len(sort_service)):
        centr = get_dist_centroid(sort_service.District[i])
        
        rad = get_inner_radius(inner_circle, sort_service[radius_feature][i], inner_rad_max, inner_rad_min)
        folium.Circle(location = centr,
                      radius = rad,
                      color ='green',
                      weight =2,
                      fill_color ='green',
                      fill_opacity = 0.9,
                      tooltip = str(sort_service[radius_feature][i]),
                      stroke = True).add_to(mapObj_dist)
        
    for i in range(len(sort_district_desc)):
        if (sort_district_desc.District[i] in list(sort_service.District)):
            f_num = sort_service[sort_service.District == sort_district_desc.District[i]].reset_index(drop=True)[radius_feature][0]
            if f_num > 100:
                color = "blue"
            else:
                color = "purple"
        else:
            f_num = 0
            color = 'red'
        try:
            f_num_tt = service_served[service_served.District == sort_district_desc.District[i]].reset_index(drop=True)[tt_feature][0]
        except:
            f_num_tt = farmer_count[farmer_count.District == sort_district_desc.District[i]].reset_index(drop=True)[tt_feature][0]
        
        tooltip = folium.Tooltip(get_tt(sort_district_desc.District[i], f"{int(int(sort_district_desc.area[i])):,}",
                                        f"{int(sort_district_desc.Total_firm_holdings_no[i]):,}",tt_crp,
                                        f"{int(sort_district_desc.Temporary_crops_net_area[i]):,}",
                                        f"{int(sort_district_desc[tt_crp][i]):,}", f"{int(f_num):,}", inner_circle, 
                                        tt_feature, f_num_tt, round((f_num/f_num_tt*100),3)))


        centr = get_dist_centroid(sort_district_desc.District[i])
        
        shapesLayer = circle_plot(centr, sort_district_desc.Total_firm_holdings_no[i]**1.025,
                                  outer_rad_max, outer_rad_min, color, color, 0.01, tooltip)
        shapesLayer.add_to(mapObj_dist)
    
    return mapObj_dist

In [33]:
# init('Advisory_Farmer', district_desc, farmer_count, service_served, 'All', 'All', 1)